In [505]:
import os 
import tarfile
from six.moves import urllib
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

DOWNLOAD_ROOT = "https://spamassassin.apache.org/old/publiccorpus"
SPAM_PATH = os.path.join("datasets", "spam/")
HAM_PATH = os.path.join("datasets", "ham/")
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam_2.tar.bz2"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham_2.tar.bz2"

In [633]:
class EmailParser(): # Could be better, still superflous information in emails.
    def __init__(self):
        self.email = ''
        
    def isHTML(self):
        with open(name, encoding='latin1') as f:
            if 'Content-Type: text/html' in f.read():
                 return True
            else:
                return False
            
    def parse(self, path_to_email):
        try:
            if(ishtml(path_to_email)):
                page = open(path_to_email, 'r', encoding='latin1')
                soup = BeautifulSoup(page, 'html.parser')

                # kill all script and style elements
                for script in soup(["script", "style"]):
                    script.extract()    # rip it out

                # get text
                text = "" + soup.get_text()

                # break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines())
                # break multi-headlines into a line each
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                # drop blank lines
                text = '\n'.join(chunk for chunk in chunks if chunk)

                self.email = text.replace('\n','')
                
            else:
                f = open(path_to_email, 'r', encoding='latin1')
                text = f.read()
                # break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines())
                # break multi-headlines into a line each
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                # drop blank lines
                text = '\n'.join(chunk for chunk in chunks if chunk)

                self.email = text.replace('\n',' ')
        except:
            print('Something went wrong parsing the email')
        return self.email



In [634]:
emailParser = EmailParser()

spams = []
hams = []
for spam_file in os.listdir(SPAM_PATH):
    spam = emailParser.parse(SPAM_PATH + spam_file)
    spams.append(spam)

    
for ham_file in os.listdir(HAM_PATH):
    ham = emailParser.parse(HAM_PATH + ham_file)
    hams.append(ham)
    
spams = pd.DataFrame(spams, dtype="string")
hams = pd.DataFrame(hams, dtype="string", index=range(500, 1900))

In [638]:
spams['spam'] = True
hams['spam'] = False
emails = spams.append(hams)
emails.rename(columns={0:'email'},
                 inplace=True)


In [663]:
# Attributes
emails

,email,spam
0,From offer-09102002-TEXT@offerclubmail.330w.co...,True
1,Return-Path: ler@lerami.lerctr.orgDelivery-Dat...,True
2,From ilug-admin@linux.ie Sat Sep 21 22:04:31 2...,True
3,From safety33o@l1.newnamedns.com Mon Aug 26 15...,True
4,From pdibeta@gmx.net Fri Sep 6 15:40:52 2002 R...,True
...,...,...
1895,From razor-users-admin@lists.sourceforge.net T...,False
1896,From webmake-talk-admin@lists.sourceforge.net ...,False
1897,From ilug-admin@linux.ie Mon Jul 22 18:12:23 2...,False
1898,From fork-admin@xent.com Fri Jul 26 07:31:07 2...,False


In [668]:
# Possibly bad imlementation of splitting, new split each time this is run
# Scikit-Learn has functions that does this + can choose random seed:
# from sklearn.model_selection import train_test_split
# trainSet, testSet = train_test_split(housing, test_size=0.2, random_state=42)
# can also split according to different DataFrame with same #rows
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(emails, test_size=0.2, random_state=42)
x_train = train_set.drop('spam',axis=1)
y_train = test_set.drop('spam', axis=1)
x_labels = train_set['spam'].copy()
y_labels = test_set['spam'].copy()

In [670]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier()